In [1]:
import json
import requests

In [2]:

def generate_with_outline(prompt, outline, model="llama3.2"):
    url = "http://localhost:11434/api/generate"
    
    formatted_prompt = f"""
    Please follow this outline in your response:
    {json.dumps(outline, indent=2)}
    
    Prompt: {prompt}
    """
    
    payload = {
        "model": model,
        "prompt": formatted_prompt,
        "stream": False
    }
    
    response = requests.post(url, json=payload)
    print(response.json())
    return response.json()["response"]

# Example usage
outline = {
    "introduction": "Brief overview",
    "main_points": [
        "Point 1",
        "Point 2",
        "Point 3"
    ],
    "conclusion": "Summary"
}

result = generate_with_outline(
    prompt="Explain the benefits of exercise",
    outline=outline
)
print(result)


{'model': 'llama3.2', 'created_at': '2024-11-15T14:36:50.442168Z', 'response': '{\n  "introduction": "Regular physical activity is a fundamental aspect of a healthy lifestyle, offering numerous benefits for overall well-being and quality of life.",\n  "main_points": [\n    "Point 1: Improved Mental Health - Exercise has been shown to reduce symptoms of anxiety and depression, while also improving mood and cognitive function.",\n    "Point 2: Enhanced Physical Health - Regular physical activity can help maintain a healthy weight, lower blood pressure, and increase strength and flexibility.",\n    "Point 3: Increased Energy and Productivity - Exercise boosts energy levels, enhances sleep quality, and improves focus and concentration, leading to increased productivity and better work performance."\n  ],\n  "conclusion": "Incorporating exercise into one\'s daily routine can lead to a significant improvement in overall health and well-being, resulting in a longer, healthier, and happier lif

In [5]:
from typing import List, Literal
from pydantic import BaseModel, Field
import outlines 

class Hobby(BaseModel):
    name: str
    description: str
    duration: int = Field(..., description="Years since having hobby")

class Character(BaseModel):
    name:str
    birth_year: int = Field(..., ge=1990, le=2025)
    hobbies: List[Hobby]
    alive: bool


lm = outlines.models.openai("llama3.2", base_url="http://localhost:11434/v1", api_key="ollama")

In [10]:
from outlines import models, generate

In [23]:
model = models.llamacpp("microsoft/Phi-3-mini-4k-instruct-gguf", "Phi-3-mini-4k-instruct-q4.gguf")

/Users/n.makdoud/miniforge3/envs/llm/lib/python3.11/site-packages/outlines/models/llamacpp.py:391: UserWarning: The pre-tokenizer in `llama.cpp` handles unicode improperly (https://github.com/ggerganov/llama.cpp/pull/5613)
Outlines may raise a `RuntimeError` when building the regex index.
To circumvent this error when using `models.llamacpp()` you may pass the argument`tokenizer=llama_cpp.llama_tokenizer.LlamaHFTokenizer.from_pretrained(<hf_repo_id>)`

  warnings.warn(


Phi-3-mini-4k-instruct-q4.gguf:   0%|          | 0.00/2.39G [00:00<?, ?B/s]

In [15]:
generator = generate.text(model)

In [26]:
from llama_cpp import Llama

In [27]:
llm = Llama.from_pretrained(
    repo_id="Qwen/Qwen2-0.5B-Instruct-GGUF",
    filename="*q8_0.gguf",
    verbose=True
)

qwen2-0_5b-instruct-q8_0.gguf:   0%|          | 0.00/531M [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 26 key-value pairs and 290 tensors from /Users/n.makdoud/.cache/huggingface/hub/models--Qwen--Qwen2-0.5B-Instruct-GGUF/snapshots/198f08841147e5196a6a69bd0053690fb1fd3857/./qwen2-0_5b-instruct-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = qwen2-0_5b-instruct
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 24
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_model_loader: - kv   4:                     qwen2.embedding_length u32              = 896
llama_model_loader: - kv   5:                  qwen2.feed_forward_length u32              = 4864
llama_model_loader: - kv   6:      

In [28]:
llm.create_chat_completion(
      messages = [
          {"role": "system", "content": "You are an assistant who perfectly describes images."},
          {
              "role": "user",
              "content": "Describe this image in detail please."
          }
      ]
)

llama_perf_context_print:        load time =     450.31 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    29 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   212 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    2164.19 ms /   241 tokens


{'id': 'chatcmpl-f58d5778-37e4-4a42-ac00-2771865e5bf5',
 'object': 'chat.completion',
 'created': 1731683163,
 'model': '/Users/n.makdoud/.cache/huggingface/hub/models--Qwen--Qwen2-0.5B-Instruct-GGUF/snapshots/198f08841147e5196a6a69bd0053690fb1fd3857/./qwen2-0_5b-instruct-q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "As an AI language model, I don't have access to specific images to describe. However, I can describe the general characteristics of an image based on its type, medium, and subject.\n\nFor example, if an image is a landscape photograph taken in a cityscape, the general characteristics may include:\n\n- The subject is a person or object in the foreground, such as a person walking down the street or a building.\n- The scene is typically bright and sunny, with natural elements like trees and buildings visible.\n- The sky, clouds, and other elements in the sky are often used to add depth to the image.\n- The subject's clothing or acce

In [32]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List

import instructor


class Character(BaseModel):
    name: str
    age: int
    fact: List[str] = Field(..., description="A list of facts about the character")


# enables `response_model` in create call
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

resp = client.chat.completions.create(
    model="llama3.2",
    messages=[
        {
            "role": "user",
            "content": "Tell me about the Harry Potter",
        }
    ],
    response_model=Character,
)
print(resp.model_dump_json(indent=2))

{
  "name": "Harry Potter",
  "age": 11,
  "fact": [
    "Wears number 7.",
    "Can perform a perfect wand movement.",
    "Enjoys treacle fudge and pumpkin pasties."
  ]
}


In [34]:
from pydantic import BaseModel, Field
from typing import Optional

class ConventionalCommit(BaseModel):
    type: str = Field(
        ...,
        description="The type of change (e.g., feat, fix, chore, refactor).",
        pattern="^(feat|fix|chore|refactor|test|docs|style|perf|ci|build|revert)$",
    )
    scope: Optional[str] = Field(
        None,
        description="The scope of the change (e.g., a specific module or component)."
    )
    description: str = Field(
        ...,
        description="A short description of the change.",
        max_length=72
    )
    body: Optional[str] = Field(
        None,
        description="Detailed explanation of the change."
    )
    footer: Optional[str] = Field(
        None,
        description="Additional information (e.g., breaking changes or issues fixed)."
    )

    @property
    def formatted(self) -> str:
        """
        Returns the conventional commit in the correct format.
        """
        parts = [f"{self.type}{f'({self.scope})' if self.scope else ''}: {self.description}"]
        if self.body:
            parts.append(f"\n\n{self.body}")
        if self.footer:
            parts.append(f"\n\n{self.footer}")
        return "".join(parts)

# Example usage
example_commit = ConventionalCommit(
    type="feat",
    scope="auth",
    description="Add OAuth2 login support",
    body="This change integrates OAuth2 login with Google and Facebook.",
    footer="BREAKING CHANGE: Updated login API; old tokens are invalidated."
)

print(example_commit.formatted)


feat(auth): Add OAuth2 login support

This change integrates OAuth2 login with Google and Facebook.

BREAKING CHANGE: Updated login API; old tokens are invalidated.
